In [11]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.embeddings import HuggingFaceHubEmbeddings,HuggingFaceEmbeddings
from langchain.llms import CTransformers,LlamaCpp
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA,ConversationalRetrievalChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate



In [2]:
DB_FAISS_PATH = "vectorestores/db_faiss"

In [3]:
#load the data 
loader = DirectoryLoader("data/",glob="*.pdf",loader_cls=PyPDFLoader)
documents = loader.load()

In [4]:
#split the text into chunks 

text_Splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
text_chunks = text_Splitter.split_documents(documents=documents)


In [5]:
#create the embeddings 

embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2",model_kwargs={'device':'cpu'})



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\dhanu\.conda\envs\llm\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dhanu\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
#create a vectore store 

vectore_store = FAISS.from_documents(text_chunks,embeddings)
vectore_store.save_local(DB_FAISS_PATH)

In [12]:
#prompt template 
custom_prompt_template = """ 
    You are AI medical assisstant and you have the following piece of information to answer the user's questopm and with the informations answer the user question ,don't hallucinate with the answers.
    Context :{}
    Question :{question}

    Only return the helpful answers 
"""

In [14]:
prompt = PromptTemplate(template=custom_prompt_template,input_variables=['context','question'])


In [7]:
#create llm 

llm  = CTransformers(model="llama-2-7b-chat.ggmlv3.q8_0.bin",model_type="llama",
                     config = {'max_new_tokens':128,'temperature':0.01})


In [8]:
memory = ConversationBufferMemory(memory_key="chat_history",return_messages = True)


In [18]:
chain = ConversationalRetrievalChain.from_llm(llm=llm,chain_type="stuff",
                                              retriever=vectore_store.as_retriever(search_kwargs={"k":2}),
                                              memory = memory)

In [20]:
# Function to handle user input and get the response
def conversation_chat(query):
    result = chain({"question": query, "chat_history": chat_history})
    chat_history.append((query, result["answer"]))
    return result["answer"]

# Initialize chat history
chat_history = []


In [21]:

# Example usage
user_input = input("Question: ")
response = conversation_chat(user_input)
print(f"Response: {response}")


# Continue taking user input and generating responses
while True:
    user_input = input("Question: ")
    if user_input.lower() == "exit":
        break
    response = conversation_chat(user_input)
    print(f"Response: {response}")

c:\Users\dhanu\.conda\envs\llm\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response:  Epilepsy is a neurological disorder characterized by recurrent seizures. These seizures are caused by abnormal electrical activity in the brain, and can vary widely in severity, frequency, and impact on daily life. There are many different types of epilepsy, each with its own unique set of symptoms and characteristics. Some people with epilepsy may experience only a few seizures throughout their lifetime, while others may have seizures that occur more frequently or persist over time. Epilepsy can affect anyone, regardless of age, gender, or ethnic
Response:  Epilepsy is a neurological disorder characterized by recurrent seizures. There are many different types of epilepsy, including:

1. Generalized epilepsy: This type of epilepsy affects both sides of the brain and can cause seizures that involve loss of consciousness, convulsions, and muscle stiffness. Examples include generalized tonic-clonic seizures and absence seizures.
2. Focal epilepsy: This type of epilepsy affects 